# Workbench

## Load the data

In [15]:
from sklearn import datasets
from sklearn import preprocessing

#X, y = datasets.fetch_openml('mnist_784', return_X_y=True)
X, y = datasets.load_digits(return_X_y=True)
labels = sorted(set(y))
X = preprocessing.scale(X)

In [16]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=42)
X_fit, X_val, y_fit, y_val = model_selection.train_test_split(X_train, y_train, random_state=42)

## A first random tree

In [17]:
import myriade
from sklearn import linear_model

clf = linear_model.LogisticRegression()
rand = myriade.multiclass.RandomBalancedHierarchyClassifier(clf)
rand.fit(X_fit, y_fit)
rand.tree_

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 7.0.2 (20221119.0110)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 
<!-- 1 -->
 
 1 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 10 -->
 
 10 
 
 
<!-- 0--10 -->
 
 0--10 
 
 
<!-- 2 -->
 
 2 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 7 -->
 
 7 
 
 
<!-- 1--7 -->
 
 1--7 
 
 
<!-- 3 -->
 
 3 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 6 -->
 
 6 
 
 7 
 
<!-- 2--6 -->
 
 2--6 
 
 
<!-- 4 -->
 
 4 
 
 1 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 5 -->
 
 5 
 
 8 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 8 -->
 
 8 
 
 6 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 9 -->
 
 9 
 
 0 
 
<!-- 7--9 -->
 
 7--9 
 
 
<!-- 11 -->
 
 11 
 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 16 -->
 
 16 
 
 
<!-- 10--16 -->
 
 10--16 
 
 
<!-- 12 -->
 
 12 
 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 15 -->
 
 15 
 
 9 
 
<!-- 11--15 -->
 
 11--15 
 
 
<!-- 13 -->
 
 13 
 
 2 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 14 -->
 
 14 
 
 5 
 
<!-- 12--14 -->
 
 12--14 
 
 
<!-- 17 -->
 
 17 
 
 3 
 
<!-- 16--17 -->
 
 16--17 
 
 
<!-- 18 -->
 
 18 
 
 4 
 
<!-- 16--18 -->
 
 16--18

Evaluate accuracy on the test set.

In [18]:
from sklearn import metrics

y_pred = rand.predict(X_test)
print(f"{metrics.accuracy_score(y_test, y_pred):.2%}")

81.78%


Predict on the validation set, in order to build the confusion matrix.

In [19]:
y_fit

array([1, 6, 9, ..., 4, 0, 4])

In [20]:
y_pred = rand.predict(X_val)
confusion = metrics.confusion_matrix(y_val, y_pred, labels=labels)
confusion = pd.DataFrame(confusion, index=labels, columns=labels).sort_index(axis=1).sort_index(axis=0)
confusion

,0,1,2,3,4,5,6,7,8,9
0,30,0,2,0,0,3,0,0,0,0
1,0,34,1,1,4,0,0,0,0,1
2,1,5,21,1,0,0,0,0,2,0
3,0,0,0,34,0,1,0,0,3,0
4,0,3,0,0,28,0,4,2,0,0
5,0,0,0,0,0,29,0,1,5,0
6,0,0,0,0,0,0,39,0,0,0
7,0,0,0,0,1,0,0,21,0,2
8,0,4,1,1,0,3,0,0,20,3
9,0,1,1,0,0,0,0,0,1,23


In [32]:
cm = confusion.to_numpy()
cm[:, -1]

array([ 0,  1,  0,  0,  0,  0,  0,  2,  3, 23])

In [33]:
cm[-1, :]

array([ 0,  1,  1,  0,  0,  0,  0,  0,  1, 23])

In [36]:
base_model = myriade.multiclass.RandomBalancedHierarchyClassifier(
    classifier=linear_model.LogisticRegression(),
    seed=42
)
cv = model_selection.KFold(
    n_splits=2,
    shuffle=True,
    random_state=42
)
model = myriade.multiclass.BalancedHierarchyClassifier(
    classifier=linear_model.LogisticRegression(),
    base_model=base_model,
    cv=cv
)
model.fit(X_train, y_train)
print(f"{model.score(X_test, y_test):.2%}")

94.89%


## Learning a (better) tree structure from the confusion matrix

In [102]:
import warnings
import numpy as np

def make_tree_from_pairs(pairs):
    return myriade.Branch(
        make_tree_from_pairs(pairs[0]) if isinstance(pairs[0], tuple) else pairs[0],
        make_tree_from_pairs(pairs[1]) if isinstance(pairs[1], tuple) else pairs[1]
    )


def pair_labels(confusion):

    labels = confusion.index.tolist()

    errors = np.triu(1 + confusion + confusion.T, k=1)
    
    # Here we find the pairs of labels that are most confused with each other. We keep going until
    # there are no more pairs to compare.
    pairs_idx = []
    while errors.any():
        i, j = np.unravel_index(np.argmax(errors), errors.shape)
        pairs_idx.append((i, j))
        errors[i, :] = 0
        errors[:, i] = 0
        errors[j, :] = 0
        errors[:, j] = 0
    pairs = [(labels[i], labels[j]) for i, j in pairs_idx]

    # If the number of labels is odd, there will be one label left over. We call this an orphan.
    orphans_idx = list(set(range(len(labels))) - set(i for i, _ in pairs_idx) - set(j for _, j in pairs_idx))
    orphans = [labels[idx] for idx in orphans_idx]

    # We now create a new confusion matrix that combines the pairs of labels.
    confusion_arr = (
        confusion.iloc[[p[0] for p in pairs_idx] + orphans_idx, [p[0] for p in pairs_idx] + orphans_idx].to_numpy() +
        confusion.iloc[[p[0] for p in pairs_idx] + orphans_idx, [p[1] for p in pairs_idx] + orphans_idx].to_numpy() +
        confusion.iloc[[p[1] for p in pairs_idx] + orphans_idx, [p[0] for p in pairs_idx]+ orphans_idx].to_numpy() +
        confusion.iloc[[p[1] for p in pairs_idx] + orphans_idx, [p[1] for p in pairs_idx]+ orphans_idx].to_numpy()
    )

    # HACK: If there are orphans, we need to divide the corresponding rows and columns by 2 to
    # avoid double counting.
    if orphans:
        confusion_arr[-1, :] = confusion_arr[-1, :] / 2
        confusion_arr[:, -1] = confusion_arr[:, -1] / 2

    confusion = pd.DataFrame(confusion_arr, index=pairs + orphans, columns=pairs + orphans)

    # Termination condition
    if len(confusion.columns) == 1:
        return make_tree_from_pairs(pairs[0])

    return pair_labels(confusion)

confusion = metrics.confusion_matrix(y_val, y_pred, labels=labels)
confusion = pd.DataFrame(confusion, index=labels, columns=labels).sort_index(axis=1).sort_index(axis=0)
new_tree = pair_labels(confusion)
new_tree

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 7.0.2 (20221119.0110)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 
<!-- 1 -->
 
 1 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 16 -->
 
 16 
 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 2 -->
 
 2 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 9 -->
 
 9 
 
 
<!-- 1--9 -->
 
 1--9 
 
 
<!-- 3 -->
 
 3 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 6 -->
 
 6 
 
 
<!-- 2--6 -->
 
 2--6 
 
 
<!-- 4 -->
 
 4 
 
 3 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 5 -->
 
 5 
 
 8 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 7 -->
 
 7 
 
 0 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 8 -->
 
 8 
 
 1 
 
<!-- 6--8 -->
 
 6--8 
 
 
<!-- 10 -->
 
 10 
 
 
<!-- 9--10 -->
 
 9--10 
 
 
<!-- 13 -->
 
 13 
 
 
<!-- 9--13 -->
 
 9--13 
 
 
<!-- 11 -->
 
 11 
 
 4 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 12 -->
 
 12 
 
 7 
 
<!-- 10--12 -->
 
 10--12 
 
 
<!-- 14 -->
 
 14 
 
 2 
 
<!-- 13--14 -->
 
 13--14 
 
 
<!-- 15 -->
 
 15 
 
 5 
 
<!-- 13--15 -->
 
 13--15 
 
 
<!-- 17 -->
 
 17 
 
 6 
 
<!-- 16--17 -->
 
 16--17 
 
 
<!-- 18 -->
 
 18 
 
 9 
 
<!-- 16--18 -->
 
 16--18

In [103]:
better = myriade.multiclass.ManualHierarchyClassifier(clf, new_tree)
better.fit(X_fit, y_fit)
y_pred = rand.predict(X_test)
print(f"{metrics.accuracy_score(y_test, y_pred):.2%}")

94.44%


In [13]:
import myriade
from sklearn import linear_model, metrics

clf = linear_model.LogisticRegression()
balanced = myriade.multiclass.BalancedHierarchyClassifier(clf)
balanced.fit(X_fit, y_fit)
y_pred = balanced.predict(X_test)
print(f"{metrics.accuracy_score(y_test, y_pred):.2%}")

93.78%


In [14]:
confusion

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/8q/xqx16rw14bl7rqyybl7zk8sh0000gn/T/ipykernel_44317/3610483612.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/8q/xqx16rw14bl7rqyybl7zk8sh0000gn/T/ipykernel_44317/3610483612.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'confusion' is not defined